In [89]:
import mne
import numpy as np

Open BDF file/ Remove unused BioSemi channels

In [102]:
raw = mne.io.read_raw_bdf("user_18_session_1.bdf", preload = True) 

raw_data = raw.copy()
raw_data.pick(raw_data.ch_names[:32])

Extracting EDF parameters from c:\Users\irakl\Desktop\Bachelor Thesis\Raw Data\user_18_session_1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 401407  =      0.000 ...   196.000 secs...


<RawEDF | user_18_session_1.bdf, 32 x 401408 (196.0 s), ~98.0 MiB, data loaded>

Get Events through trigger(Status) channel

In [103]:
events = mne.find_events(raw, stim_channel = 'Status', shortest_event = 1)

Trigger channel Status has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
15 events found on stim channel Status
Event IDs: [2]


Deleta data recorded before the experiment starts

In [104]:
first_stim = events[events[:, 2] == 2][0, 0] #find the first code 2 event in frames
last_stim = events[events[:, 2] == 2][-1, 0]
first_stim_time = raw_data.times[first_stim]
last_stim_time = raw_data.times[last_stim]
start_time = max(0, first_stim_time - 3.0)
end_time = min(raw_data.times[-1], last_stim_time + 1.0  # stim duration
                                                  + 2.0  # extra time until resting state screen closes
                                                  + 1.0) # extra time to be sure    

raw_data.crop(tmin = start_time, tmax = end_time)

<RawEDF | user_18_session_1.bdf, 32 x 350622 (171.2 s), ~85.6 MiB, data loaded>

Drift and high-frequency removal

In [106]:
raw_data.filter(l_freq = 0.2, h_freq = None)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.10 Hz)
- Filter length: 33793 samples (16.500 s)



<RawEDF | user_18_session_1.bdf, 32 x 350622 (171.2 s), ~85.6 MiB, data loaded>

Spectrum Analysis(Remove power line noise)

In [107]:
fig = raw_data.compute_psd(fmax = 250).plot(
        average = True, amplitude = False, picks = "data", exclude = "bads"
)
fig.savefig("psd_plot.png", dpi = 300)

Effective window size : 1.000 (s)
Plotting power spectral density (dB=True).


c:\Users\irakl\AppData\Local\Programs\Python\Python313\Lib\site-packages\mne\viz\utils.py:158: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  (fig or plt).show(**kwargs)


In [109]:
freqs = (50, 100, 150, 200, 250)
raw_data.notch_filter(freqs = freqs, method = "spectrum_fit")
fig = raw_data.compute_psd(fmax = 250).plot(
        average = True, amplitude = False, picks = "data", exclude = "bads"
)
fig.savefig("psd_after_notch_plot.png", dpi = 300)

Filtering raw data in 1 contiguous segment
Removed notch frequencies (Hz):
     50.00 : 1056 windows
    100.00 : 1056 windows
    150.00 : 1056 windows
    200.00 : 1056 windows
    249.00 : 1056 windows
    250.00 : 1056 windows
    251.00 : 1056 windows
Effective window size : 1.000 (s)
Plotting power spectral density (dB=True).


c:\Users\irakl\AppData\Local\Programs\Python\Python313\Lib\site-packages\mne\viz\utils.py:158: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  (fig or plt).show(**kwargs)


Artifact removal

Resample

In [ ]:
raw_data.filter(h_freq = 70.0)
raw_data.resample(256)